# 3. Feature Extraction, Transformations and Cleansing: 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
%matplotlib inline
matplotlib.style.use("ggplot")

In [2]:
pd.set_option('display.max_columns', None)

## 3.1. Data Quality: nans, wrong data, etc.

### File loading and examine:

#### 1. Vamos a utilizar los data sets desde el 2000 hasta el 2014. Creamos un algoritmo que sea capaz de cargar y concatenar todos los archivos a la vez, previo a  empezar la limpieza

In [3]:
import glob

In [4]:
path =r'../tennis_atp-master/Prueba' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

#### 2. Ahora aplico el proceso de limpieza que hemos dictaminado como necesario:

In [5]:
# 1. Take out matchs from Davies Cup:
framesD = frame[frame['tourney_level'] != 'D']
# 2. Take out matchs from Challenge CUP:
framesDC = framesD[framesD['tourney_level'] != 'C']
# 3.1. Take out marchs'W/O' (Walk Over):
framesDCWO = framesDC[framesDC['score'] != 'W/O']
# 3.2. Take out marchs' W/O' (Walk Over):
framesDCWO = framesDCWO[framesDCWO['score'] != ' W/O']
# 4. Take out matchs with high volume of NaNs:
framesDCWO2 = framesDCWO.dropna(axis=0, thresh=30)

In [6]:
x = frame.shape
y = framesDCWO2.shape
print x
print y

(47221, 49)
(41796, 49)


#### La limpieza ha supuesto la eliminacion de 5.425 partidos, un 11,5% del total.

#### 3. New Feature creation: Retired. This feature contemple aquellos partidos que no terminaron

In [7]:
Retired = []
for row in framesDCWO2['score']:
    if 'RET' in row:
        # Append 1
        Retired.append(1)
    # else, if not append 0,
    else:
        Retired.append(0)
# Create a column from the list
framesDCWO2['Retired'] = Retired

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
framesDCWO2[framesDCWO2['Retired'] == 1 ].shape

(1327, 50)

#### Clasificados como Retired hay 1.327 partidos, lo que supone un 3,2%. Sabemos que hay algunos más que no han sido clasificados como Retired y que tendremos que buscar mas adelante (partido dura menos sets de los programados)

#### 4. Solve *_ht (height) problem:

In [ ]:
for row in framesDCWO2['loser_name']:
    if 'Alejandro Gonzalez' in row:
        # Append 1
        framesDCWO2['loser_ht'] = 191
    # else, do nothing,
    else:
        pass

In [23]:
### Terminarlo sistematizarlo y cambiar Alejandro Gonzalez y 191 por un diccionario

In [25]:
Names = {'Alejandro Gonzalez': 191, 'Alexandre Sidorenko': 185}

In [26]:
for row in framesDCWO2['loser_name']:
    for k, v in Names.items():
        if k in row:
        # Append key
            framesDCWO2['loser_ht'] = v
    # else, do nothing,
    else:
        pass

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
framesDCWO2[framesDCWO2['loser_name'] == 'Alejandro Gonzalez'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired
1511,2013-6718,Bogota,Hard,28,A,20130715,7,105173,5.0,NaN,Adrian Mannarino,L,183.0,FRA,25.043121,74.0,695.0,105292,NaN,NaN,Alejandro Gonzalez,R,191,COL,24.432580,159.0,331.0,6-4 0-6 6-4,3,R32,124.0,2.0,2.0,97.0,60.0,35.0,16.0,13.0,11.0,16.0,7.0,6.0,86.0,47.0,34.0,16.0,13.0,6.0,10.0,0
78,2014-403,Miami Masters,Hard,96,M,20140319,52,104755,9.0,NaN,Richard Gasquet,R,185.0,FRA,27.745380,9.0,2905.0,105292,NaN,NaN,Alejandro Gonzalez,R,191,COL,25.103354,83.0,632.0,7-6(7) 6-4,3,R64,121.0,7.0,0.0,62.0,41.0,30.0,15.0,11.0,0.0,2.0,4.0,2.0,106.0,58.0,41.0,22.0,11.0,10.0,13.0,0
211,2014-506,Buenos Aires,Clay,32,A,20140210,8,104871,8.0,NaN,Jeremy Chardy,R,188.0,FRA,26.995209,45.0,940.0,105292,NaN,NaN,Alejandro Gonzalez,R,191,COL,25.007529,78.0,676.0,6-4 7-5,3,R32,102.0,5.0,2.0,67.0,35.0,24.0,18.0,11.0,4.0,6.0,3.0,1.0,82.0,51.0,30.0,15.0,11.0,10.0,14.0,0
254,2014-416,Rome Masters,Clay,56,M,20140511,20,103565,NaN,Q,Stephane Robert,R,185.0,FRA,33.984942,85.0,629.0,105292,NaN,Q,Alejandro Gonzalez,R,191,COL,25.256674,79.0,647.0,1-6 6-4 6-1,3,R64,96.0,7.0,2.0,72.0,47.0,35.0,13.0,12.0,1.0,3.0,5.0,1.0,77.0,53.0,34.0,13.0,12.0,2.0,5.0,0
370,2014-6718,Bogota,Hard,28,A,20140714,23,103285,4.0,NaN,Radek Stepanek,R,185.0,CZE,35.627652,39.0,1085.0,105292,6.0,NaN,Alejandro Gonzalez,R,191,COL,25.429158,74.0,686.0,6-4 6-4,3,QF,80.0,8.0,2.0,60.0,43.0,36.0,8.0,10.0,3.0,3.0,7.0,2.0,62.0,40.0,27.0,11.0,10.0,1.0,3.0,0


In [28]:
framesDCWO2[framesDCWO2['loser_name'] == 'Alexandre Sidorenko'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired
1603,2006-496,Marseille,Hard,32,A,20060213,11,105095,NaN,LL,Evgeny Korolev,R,185.0,KAZ,17.998631,188.0,206.0,105098,NaN,WC,Alexandre Sidorenko,R,191,FRA,17.987680,1291.0,3.0,2-6 6-4 6-1,3,R32,112.0,8.0,7.0,82.0,41.0,35.0,20.0,13.0,6.0,8.0,5.0,2.0,78.0,34.0,22.0,24.0,12.0,1.0,4.0,0
361,2007-520,Roland Garros,Clay,128,G,20070528,18,102381,NaN,NaN,Werner Eschauer,R,188.0,AUT,33.086927,93.0,442.0,105098,NaN,WC,Alexandre Sidorenko,R,191,FRA,19.271732,304.0,115.0,6-3 3-6 6-3 5-4 RET,5,R128,188.0,2.0,4.0,124.0,78.0,52.0,22.0,18.0,5.0,10.0,3.0,0.0,135.0,86.0,50.0,23.0,18.0,15.0,22.0,1
533,2008-308,Munich,Clay,32,A,20080428,10,101962,NaN,Q,Younes El Aynaoui,R,193.0,MAR,36.626968,295.0,120.0,105098,NaN,Q,Alexandre Sidorenko,R,191,FRA,20.191650,317.0,110.0,6-4 7-6(6),3,R32,92.0,14.0,3.0,80.0,51.0,40.0,15.0,11.0,1.0,1.0,3.0,3.0,64.0,41.0,33.0,12.0,11.0,1.0,2.0,0
363,2009-321,Stuttgart,Clay,32,A,20090713,27,103812,4.0,NaN,Victor Hanescu,R,198.0,ROU,27.978097,33.0,1361.0,105098,NaN,LL,Alexandre Sidorenko,R,191,FRA,21.399042,193.0,351.0,7-6(2) 6-4,3,QF,84.0,6.0,1.0,66.0,44.0,38.0,13.0,11.0,1.0,1.0,2.0,5.0,67.0,41.0,33.0,14.0,11.0,3.0,4.0,0
2130,2009-520,Roland Garros,Clay,128,G,20090525,28,103498,20.0,NaN,Marat Safin,R,193.0,RUS,29.325120,22.0,1845.0,105098,NaN,WC,Alexandre Sidorenko,R,191,FRA,21.264887,145.0,527.0,6-4 6-4 6-4,5,R128,137.0,6.0,2.0,92.0,52.0,38.0,24.0,15.0,5.0,7.0,4.0,5.0,108.0,54.0,39.0,20.0,15.0,7.0,12.0,0


## OK este es el correcto!!!

In [9]:
Names = {'Alejandro Gonzalez': 191, 'Alexandre Sidorenko': 185}

In [13]:
framesDCWO2['loser_ht'] = framesDCWO2['loser_ht'].fillna(framesDCWO2['loser_name'].map(Names))

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
framesDCWO2[framesDCWO2['loser_name'] == 'Alejandro Gonzalez'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired
1511,2013-6718,Bogota,Hard,28,A,20130715,7,105173,5.0,NaN,Adrian Mannarino,L,183.0,FRA,25.043121,74.0,695.0,105292,NaN,NaN,Alejandro Gonzalez,R,191.0,COL,24.432580,159.0,331.0,6-4 0-6 6-4,3,R32,124.0,2.0,2.0,97.0,60.0,35.0,16.0,13.0,11.0,16.0,7.0,6.0,86.0,47.0,34.0,16.0,13.0,6.0,10.0,0
78,2014-403,Miami Masters,Hard,96,M,20140319,52,104755,9.0,NaN,Richard Gasquet,R,185.0,FRA,27.745380,9.0,2905.0,105292,NaN,NaN,Alejandro Gonzalez,R,191.0,COL,25.103354,83.0,632.0,7-6(7) 6-4,3,R64,121.0,7.0,0.0,62.0,41.0,30.0,15.0,11.0,0.0,2.0,4.0,2.0,106.0,58.0,41.0,22.0,11.0,10.0,13.0,0
211,2014-506,Buenos Aires,Clay,32,A,20140210,8,104871,8.0,NaN,Jeremy Chardy,R,188.0,FRA,26.995209,45.0,940.0,105292,NaN,NaN,Alejandro Gonzalez,R,191.0,COL,25.007529,78.0,676.0,6-4 7-5,3,R32,102.0,5.0,2.0,67.0,35.0,24.0,18.0,11.0,4.0,6.0,3.0,1.0,82.0,51.0,30.0,15.0,11.0,10.0,14.0,0
254,2014-416,Rome Masters,Clay,56,M,20140511,20,103565,NaN,Q,Stephane Robert,R,185.0,FRA,33.984942,85.0,629.0,105292,NaN,Q,Alejandro Gonzalez,R,191.0,COL,25.256674,79.0,647.0,1-6 6-4 6-1,3,R64,96.0,7.0,2.0,72.0,47.0,35.0,13.0,12.0,1.0,3.0,5.0,1.0,77.0,53.0,34.0,13.0,12.0,2.0,5.0,0
370,2014-6718,Bogota,Hard,28,A,20140714,23,103285,4.0,NaN,Radek Stepanek,R,185.0,CZE,35.627652,39.0,1085.0,105292,6.0,NaN,Alejandro Gonzalez,R,191.0,COL,25.429158,74.0,686.0,6-4 6-4,3,QF,80.0,8.0,2.0,60.0,43.0,36.0,8.0,10.0,3.0,3.0,7.0,2.0,62.0,40.0,27.0,11.0,10.0,1.0,3.0,0


In [15]:
framesDCWO2[framesDCWO2['loser_name'] == 'Alexandre Sidorenko'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired
1603,2006-496,Marseille,Hard,32,A,20060213,11,105095,NaN,LL,Evgeny Korolev,R,185.0,KAZ,17.998631,188.0,206.0,105098,NaN,WC,Alexandre Sidorenko,R,185.0,FRA,17.987680,1291.0,3.0,2-6 6-4 6-1,3,R32,112.0,8.0,7.0,82.0,41.0,35.0,20.0,13.0,6.0,8.0,5.0,2.0,78.0,34.0,22.0,24.0,12.0,1.0,4.0,0
361,2007-520,Roland Garros,Clay,128,G,20070528,18,102381,NaN,NaN,Werner Eschauer,R,188.0,AUT,33.086927,93.0,442.0,105098,NaN,WC,Alexandre Sidorenko,R,185.0,FRA,19.271732,304.0,115.0,6-3 3-6 6-3 5-4 RET,5,R128,188.0,2.0,4.0,124.0,78.0,52.0,22.0,18.0,5.0,10.0,3.0,0.0,135.0,86.0,50.0,23.0,18.0,15.0,22.0,1
533,2008-308,Munich,Clay,32,A,20080428,10,101962,NaN,Q,Younes El Aynaoui,R,193.0,MAR,36.626968,295.0,120.0,105098,NaN,Q,Alexandre Sidorenko,R,185.0,FRA,20.191650,317.0,110.0,6-4 7-6(6),3,R32,92.0,14.0,3.0,80.0,51.0,40.0,15.0,11.0,1.0,1.0,3.0,3.0,64.0,41.0,33.0,12.0,11.0,1.0,2.0,0
363,2009-321,Stuttgart,Clay,32,A,20090713,27,103812,4.0,NaN,Victor Hanescu,R,198.0,ROU,27.978097,33.0,1361.0,105098,NaN,LL,Alexandre Sidorenko,R,185.0,FRA,21.399042,193.0,351.0,7-6(2) 6-4,3,QF,84.0,6.0,1.0,66.0,44.0,38.0,13.0,11.0,1.0,1.0,2.0,5.0,67.0,41.0,33.0,14.0,11.0,3.0,4.0,0
2130,2009-520,Roland Garros,Clay,128,G,20090525,28,103498,20.0,NaN,Marat Safin,R,193.0,RUS,29.325120,22.0,1845.0,105098,NaN,WC,Alexandre Sidorenko,R,185.0,FRA,21.264887,145.0,527.0,6-4 6-4 6-4,5,R128,137.0,6.0,2.0,92.0,52.0,38.0,24.0,15.0,5.0,7.0,4.0,5.0,108.0,54.0,39.0,20.0,15.0,7.0,12.0,0


#### Ahora tenemos que hacer un diccionario con todos los *_ht que faltan:

In [ ]:
framesDCWO2.isnull().sum()

In [ ]:
w_Names = {'Adrian Cruciat': 185,
 'Alejandro Gonzalez': 191,
 'Aleksandr Nedovyesov': 193,
 'Alexander Zverev': 191,
 'Alexandre Sidorenko': 185,
 'Ante Pavic': 196,
 'Augustin Gensse': 180,
 'Austin Krajicek': 188,
 'Axel Michon': 176,
 'Bastian Knittel': 180,
 'Blaz Rola': 193,
 'Borna Coric': 185,
 'Chris Eaton': 188,
 'Christian Garin': 185,
 'Christian Harrison': 180,
 'Christian Lindell': 193,
 'Chuhan Wang': 188,
 'Daniel Kosakowski': 185,
 'Daniel Munoz De La Nava': 178,
 'Dennis Novikov': 193,
 'Diego Sebastian Schwartzman': 170,
 'Dimitar Kutrovsky': 175,
 'Dominic Thiem': 185,
 'Elias Ymer': 183,
 'Emilio Gomez': 185,
 'Facundo Bagnis': 183,
 'Filip Peliwo': 178,
 'Gianluca Naso': 193,
 'Guido Pella': 185,
 'Guillermo Olaso': 175,
 'Gustavo Marcaccio': 170,
 'Henri Kontinen': 191,
 'Henri Laaksonen': 185,
 'Hiroki Kondo': 178,
 'Hiroki Moriya': 170,
 'Inigo Cervantes Huegun': 183,
 'Ivan Sergeyev': 175,
 'Jamie Baker': 183,
 'Jan Lennard Struff': 196,
 'Jaroslav Pospisil': 178,
 'Jason Kubler': 178,
 'Javier Marti': 185,
 'Jiri Vesely': 198,
 'Joao Sousa': 185,
 'Jonathan Dasnieres De Veigy': 175,
 'Jonathan Marray': 183,
 'Jorge Aguilar': 178,
 'Karen Khachanov': 198,
 'Kyle Edmund': 188,
 'Laurent Recouderc': 178,
 'Leonardo Tavares': 185,
 'Lucas Pouille': 185,
 'Luka Gregorc': 190,
 'Luke Saville': 188,
 'Marco Trungelliti': 180,
 'Martin Alund': 183,
 'Martin Fischer': 180,
 'Marton Fucsovics': 188,
 'Mate Delic': 196,
 'Mate Pavic': 193,
 'Mathieu Rodrigues': 180,
 'Matteo Viola': 185,
 'Maxime Teixeira': 188,
 'Miloslav Mecir': 196,
 'Mirza Basic': 188,
 'Nick Kyrgios': 193,
 'Nikola Milojevic': 188,
 'Nils Langer': 193,
 'Pablo Carreno Busta': 188,
 'Patrik Rosenholm': 178,
 'Pavol Cervenak': 191,
 'Peng Sun': 184,
 'Petar Jelenic': 190,
 'Philip Davydenko': 183,
 'Philipp Oswald': 201,
 'Pierre Hugues Herbert': 188,
 'Pierre Ludovic Duclos': 188,
 'Radu Albot': 175,
 'Rafael Arevalo': 180,
 'Ramkumar Ramanathan': 188,
 'Renzo Olivo': 180,
 'Rhyne Williams': 185,
 'Roberto Bautista Agut': 183,
 'Roberto Carballes Baena': 180,
 'Ruben Bemelmans': 183,
 'Samuel Groth': 193,
 'Sebastian Rieschick': 188,
 'Sergei Bubka': 188,
 'Takahiro Terachi': 174,
 'Taro Daniel': 191,
 'Thanasi Kokkinakis': 196,
 'Tim Puetz': 185,
 'Yan Bai': 185,
 'Yannick Mertens': 188,
 'Yuichi Sugita': 175,
 'Ze Zhang': 188}

In [ ]:
framesDCWO2['winner_ht'] = framesDCWO2['winner_ht'].fillna(framesDCWO2['winner_name'].map(w_Names))

In [ ]:
framesDCWO2.isnull().sum()

## 3.2. Data Transformation:

#### 1. Creating new Features: Sets

In [ ]:
# Separo los sets:
framesDCWO2['s1'], framesDCWO2['s5'] = framesDCWO2['score'].str.split(' ', 1).str
framesDCWO2['s2'], framesDCWO2['s5'] = framesDCWO2['s5'].str.split(' ', 1).str
framesDCWO2['s3'], framesDCWO2['s5'] = framesDCWO2['s5'].str.split(' ', 1).str
framesDCWO2['s4'], framesDCWO2['s5'] = framesDCWO2['s5'].str.split(' ', 1).str
# Separo los juegos:
framesDCWO2['w_s1'], framesDCWO2['l_s1'] = framesDCWO2['s1'].str.split('-', 1).str
framesDCWO2['w_s2'], framesDCWO2['l_s2'] = framesDCWO2['s2'].str.split('-', 1).str
framesDCWO2['w_s3'], framesDCWO2['l_s3'] = framesDCWO2['s3'].str.split('-', 1).str
framesDCWO2['w_s4'], framesDCWO2['l_s4'] = framesDCWO2['s4'].str.split('-', 1).str
framesDCWO2['w_s5'], framesDCWO2['l_s5'] = framesDCWO2['s5'].str.split('-', 1).str
# Quito los tiebreaks

#### 2. Eliminating Features:

In [ ]:
el framesDCWO2['s1']
del framesDCWO2['s2']
del framesDCWO2['s3']
del framesDCWO2['s4']
del framesDCWO2['s5']
# Cuando repase todo hacer reengeniere:
# df.drop(df.columns[[0, 1, 3]], axis=1)

In [24]:
framesDCWO2.columns

Index([u'tourney_id', u'tourney_name', u'surface', u'draw_size',
       u'tourney_level', u'tourney_date', u'match_num', u'winner_id',
       u'winner_seed', u'winner_entry', u'winner_name', u'winner_hand',
       u'winner_ht', u'winner_ioc', u'winner_age', u'winner_rank',
       u'winner_rank_points', u'loser_id', u'loser_seed', u'loser_entry',
       u'loser_name', u'loser_hand', u'loser_ht', u'loser_ioc', u'loser_age',
       u'loser_rank', u'loser_rank_points', u'score', u'best_of', u'round',
       u'minutes', u'w_ace', u'w_df', u'w_svpt', u'w_1stIn', u'w_1stWon',
       u'w_2ndWon', u'w_SvGms', u'w_bpSaved', u'w_bpFaced', u'l_ace', u'l_df',
       u'l_svpt', u'l_1stIn', u'l_1stWon', u'l_2ndWon', u'l_SvGms',
       u'l_bpSaved', u'l_bpFaced', u'Retired'],
      dtype='object')

## 3.3. Data Analisis:

### 3.3.1. Feature distribution: